In [1]:
import os
import pickle
from ase.db import connect
from ase.visualize import view
import networkx as nx
from copy import deepcopy

from care.rnet.networks.reaction_network import ReactionNetwork
from care.rnet.networks.surface import Surface

# with open('../scripts/C2_O-1_Co0001/rxn_net_bp.pkl', 'rb') as pickle_file:
#     content = pickle.load(pickle_file)  # dict of elementary reactions

# net = ReactionNetwork().from_dict(content)
# print(net)

In [3]:
from pickle import load

with open('../scripts/C1_O-1/intermediates.pkl', 'rb') as pickle_file:
    intermediates = load(pickle_file)
with open('../scripts/C1_O-1/reactions.pkl', 'rb') as pickle_file:    
    reactions = load(pickle_file)
with open('../scripts/C1_O-1/Co0001/adsorbed_configs.pkl', 'rb') as pickle_file:    
    configs = load(pickle_file)

In [4]:
configs

{'0104000101*': [Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH4', pbc=True, cell=[7.47363, 8.6298045846, 20.0]),
  Atoms(symbols='Co48CH

In [3]:
counter = 0
for intermediate in intermediates.values():
    if intermediate.phase == 'gas':
        counter += 1
print(counter)

48


In [4]:
#print(intermediates)
print(type(list(intermediates.values())[0]))

<class 'care.rnet.networks.intermediate.Intermediate'>


In [6]:
# check which type of elementary reactions are present and count them
r_types = {}
for i, step in enumerate(reactions):
    if step.r_type in r_types:
        r_types[step.r_type] += 1
    else:
        r_types[step.r_type] = 1

for i, step in enumerate(reactions):
    if step.r_type == 'O-OH':
        print(i, step, step.r_type)

print(r_types)





1120 0002020101*(H2O2*)+*<->0001010101*(HO*) O-OH
1122 0001020101*(HO2*)+*<->0000010101*(O*)+0001010101*(HO*) O-OH
{'C-OH': 220, 'H-O': 224, 'C-C': 210, 'C-H': 232, 'C-O': 236, 'O-OH': 2, 'O-O': 1, 'H-H': 1, 'adsorption': 50, 'rearrangement': 57}


In [ ]:
step1 = net[0]
print(step1)
step2 = net[1]
print(step2)

add = step1 + step2
print(add)

In [ ]:
step1 = net[0]
print(step1)

add = step1 * 0.5
print(add)

In [ ]:
step1 = net[1000]
print(step1)

step1.reverse()
print(step1)

In [ ]:
import pickle
with open('step_test.pkl', 'wb') as pickle_file:
    pickle.dump(step, pickle_file)

In [ ]:
intermediate = net[1340].reactants
intermediate = list(intermediate)[0]
print(intermediate)

with open('intermediate_test.pkl', 'wb') as pickle_file:
    pickle.dump(intermediate, pickle_file)

In [ ]:
graph = net.gen_graph(del_surf=True, show_steps=False)

In [ ]:
for node in graph.nodes:
    #print nC attribute
    print(graph.nodes[node]['formula'])

In [ ]:
print(graph)

In [ ]:
import networkx as nx
nx.write_gexf(graph, "c4pd111_nosteps.gexf")

In [ ]:
a = net.search_reaction(inters='0102000101*')
for step in a:
    print(step, step.r_type)

In [ ]:
for i in net:
    print(i)

In [ ]:
print(net[104])

In [ ]:
step1 = net[0]
print(step1)
step2 = net[1]
print(step2)
step3 = net[2]
print(step3)
print("TOTAL STEP")
step = 0.1 * step1 + step2 + step3
# print(step3.reactants, step3.products, step3.stoic)
print(step)

In [ ]:
step = net[104]
print(step)
inter = step.reactants[0]
print(inter)

In [ ]:
step1 = net[-1]
print(step1, step1.r_type)
step1.reverse()
print(step1, step1.r_type)


In [ ]:
net['14111g'].ads_configs

In [ ]:
net.visualize_intermediate('14111g')

In [ ]:
net['48489*'].molecule

In [ ]:
step = net[0]
print(step, step.stoic)
step *= 2
print(step, step.stoic)

In [ ]:
step1 = net[1]
print(step1)
step2 = net[2]
print(step2)
step3 = step1 + step2
print(step3)
print(step3.stoic)

In [ ]:
content

In [ ]:
rxn_net_copy = deepcopy(rxn_net)

In [ ]:
rxn_net == rxn_net_copy

In [ ]:
step = rxn_net[9]
step.draft()
# step.full_order()
# print(step)
# step.bb_order()
# print(step)

In [ ]:
step.reactants

In [ ]:
rxn_net.search_connections()

In [ ]:
rxn_net_copy.del_intermediates(['14111*'])

In [ ]:
rxn_net == rxn_net_copy

In [ ]:
rxn_net.search_reaction_by_inter(['13111*', '01011*'])

# Try global reactions identifier

In [ ]:
ngr = rxn_net.get_num_global_reactions(['10211g', '02011g'], ['14111g', '10111g', '02111g'])

In [ ]:
ngr


# delete intermediates

In [ ]:
rxn_net.del_intermediates(["12411*"])
print(rxn_net)

In [ ]:
rxn_net.reactions[0]
rxn_net.search_reaction_by_code('14011*(CH4*)+00000*(Pd48*)<->13011*(CH3*)+01011*(H*)')

# Visualization

In [ ]:
rxn_net.visualize_intermediate('02011g')

In [ ]:
for rxn in rxn_net.reactions:
    rxn.calc_reaction_barrier(bep_params=[0.55, 0.8])    
rxn_net.visualize_reaction(2, show_uncertainty=True)

# Work with graph representation

In [ ]:
graph = rxn_net.gen_graph(show_steps=False, del_surf=True)
for node in graph.nodes:
    print(node, graph.nodes[node])

In [ ]:
rxn_net.write_dotgraph(".", "test_del.png", show_steps=False)

In [ ]:
rxn_net.write_dotgraph(".", 'HIGHLIGHT_test.png', del_surf=True,  show_steps=False, highlight=x)

# interactive plotly graph

In [ ]:
graph = rxn_net.gen_graph(del_surf=True, show_steps=False, highlight=x)

In [ ]:
import plotly.graph_objects as go

def gen_interactive_graph(graph: nx.DiGraph):
    pos = nx.kamada_kawai_layout(graph)
    nx.set_node_attributes(graph, pos, 'pos')
    edge_x = []
    edge_y = []
    G = graph
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
    
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')
    
    node_x = [pos[0] for pos in list(nx.get_node_attributes(G,'pos').values())]
    node_y = [pos[1] for pos in list(nx.get_node_attributes(G,'pos').values())]
    
    hover_texts = []
    for node in G.nodes(data=True):
        img_path = node[1]['fig_path']
        formula = node[1]['formula']
        hover_text = f"""<
                <TABLE BORDER="0" CELLBORDER="0" CELLSPACING="0">
                <TR>
                <TD><IMG SRC="{img_path}"/></TD>
                </TR>
                <TR>
                <TD>{formula}</TD>
                </TR>
                </TABLE>>"""
        hover_texts.append(hover_text)
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        hovertext=hover_texts,
        marker=dict(
            showscale=True,
            size=30,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            )
        )
    )
    
    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(f"# of connections: {len(adjacencies[1])}")
    
    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text
    
    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0,l=0,r=0,t=0),
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False))
                    )
    fig.show()
    

In [ ]:
gen_interactive_graph(graph)

In [ ]:
import networkx as nx
from collections import deque

def constrained_shortest_path(graph, source, target, intermediates_to_avoid):
    visited = set()
    queue = deque([(source, [source])])  # Each element of the queue is a tuple (node, path_so_far)

    while queue:
        current_node, path_so_far = queue.popleft()

        if current_node == target:
            return path_so_far

        visited.add(current_node)

        for neighbor in graph.neighbors(current_node):
            # Skip nodes that are not elementary reactions
            if 'type' in graph.nodes[neighbor] and graph.nodes[neighbor]['type'] != 'elementary_reaction':
                continue

            # Skip intermediates to avoid
            if neighbor in intermediates_to_avoid:
                continue
            
            if neighbor not in visited:
                queue.append((neighbor, path_so_far + [neighbor]))
                visited.add(neighbor)
                
    return None  # return None if no such path exists


In [ ]:
import networkx as nx
from collections import deque

# Define your graph here
G = nx.Graph()
G.add_edges_from([
    ("A", "R1"),
    ("R1", "B"),
    ("B", "R2"),
    ("R2", "C"),
    ("B", "R3"),
    ("R3", "D"),
])

# Annotate nodes with their types
for node in ["A", "B", "C", "D"]:
    G.nodes[node]['type'] = 'intermediate'
for node in ["R1", "R2", "R3"]:
    G.nodes[node]['type'] = 'elementary_reaction'

def custom_shortest_path(rxn_net, graph, source, target):
    queue = deque([(source, [])])
    graph = graph.to_undirected()
    intermediates_visited = set()
    reactions_visited = set()
    while queue:
        current_node, path_so_far = queue.popleft()
        
        # Check if the target node is reached
        if current_node == target:
            return path_so_far

        intermediates_visited.add(current_node)

        for step in graph.neighbors(current_node):
            if step in reactions_visited:
                continue

            # Update the path based on the type of the neighbor
            inters = list(rxn_net.reactions[step].reactants) + list(rxn_net.reactions[step].products)
            unvisited_inters = [inter for inter in inters if inter not in intermediates_visited]
            if len(unvisited_inters) != 0:
                continue
            new_path = path_so_far + [step] if graph.nodes[step]['category'] not in ('ads', 'sur', 'gas') else path_so_far

            queue.append((step, new_path))
            visited.add(neighbor)

    return None  # Path not found

def custom_shortest_path(net, graph, source, target):
    visited = set()
    visited_inters = {'000000*', '010101*'}
    queue = deque([(source, [])])

    while queue:
        current_node, path_so_far = queue.popleft()
        
        # Check if the target node is reached
        if current_node == target:
            return path_so_far

        visited.add(current_node)
        if graph.nodes[current_node]['category'] in ('ads', 'sur', 'gas'):
            visited_inters.add(current_node)

        for neighbor in graph.neighbors(current_node):
            # Skip if already visited
            if neighbor in visited:
                continue

            if graph.nodes[neighbor]['category'] not in ('ads', 'sur', 'gas'):
                index = rxn_net.reactions[neighbor].index
                inters = list(net.reactions[index].reactants) + list(net.reactions[index].products)
                if not all([inter in visited_inters for inter in inters]):
                    continue

            # Update the path based on the type of the neighbor
            new_path = path_so_far + [neighbor] if graph.nodes[neighbor]['type'] == 'elementary_reaction' else path_so_far

            queue.append((neighbor, new_path))
            visited.add(neighbor)

    return None  # Path not found

def shortest_path_sm(graph, source, target):
    # select all nodes that are not intermediates
    visited_intermediates = set()
    rxn_nodes = [node for node in graph.nodes if graph.nodes[node]['category'] not in ('ads', 'sur', 'gas')]
    break_condition = lambda node: target in node
    cc_condition = lambda node: all([intermediate in visited_intermediates for intermediate in node])




In [ ]:
nx_graph = rxn_net.graph

In [ ]:
x = custom_shortest_path(rxn_net, nx_graph, '102101g', '141101g')
x

In [ ]:
rxn_net.reactions

# Other

In [ ]:
for inter in rxn_net.intermediates.values():
    if inter.closed_shell:
        print(inter.smiles)

In [ ]:
rxn_net.add_eley_rideal('101101g', '001101*', '102101*')
print(rxn_net)

In [ ]:
print(len(rxn_net.reactions))

In [ ]:
print(len(rxn_net.reactions))

In [ ]:
counter  =0 
for reaction in rxn_net.reactions:
    counter += 1
    print(counter, reaction.code, reaction.components, reaction.r_type)

In [ ]:
for reaction in rxn_net.reactions:
    print(reaction.components)
    for component in reaction.components:
        for inter in component:
            print(inter.code)

In [ ]:
print(len(rxn_net.intermediates))

In [ ]:
closed_shell_atoms = []
for inter in rxn_net.intermediates.values():
    if inter.closed_shell == True:
        closed_shell_atoms.append(inter.molecule)
print(len(closed_shell_atoms))

In [ ]:
closed_shell_atoms = []
for inter in rxn_net.intermediates.values():
    if inter.closed_shell == True:
        closed_shell_atoms.append(inter.molecule)
print(len(closed_shell_atoms))

In [ ]:
closed_shell_atoms[0].get_chemical_symbols().count("H")

In [ ]:
view(closed_shell_atoms)

In [ ]:
y = rxn_net.gen_graph()
# y.remove_node("")
for node in y.nodes(data=True):
    print(node)
print(y)

In [ ]:
rxn_net.surface.facet

In [ ]:
rxn_net.write_dotgraph(".", 'OLIV_test.png', del_surf=True)

Look for intermediates with specified composition

In [ ]:
rxn_net.search_inter_by_elements({'C':1, 'H':2, 'O':2})

Look for all elementary steps involving a specific intermediate

In [ ]:
rxn_net.search_ts(["222101"])

In [ ]:
types = []
for reaction in rxn_net.reactions:
    types.append(reaction.r_type)
print(set(types))

In [ ]:
types = []
for inter in rxn_net.intermediates.values():
    types.append(inter.phase)
print(set(types))

# Try PyEnergyDiagrams>

In [ ]:
import matplotlib.pyplot as plt
from energydiagram import ED
diagram = ED()
diagram.add_level(0,'Separated Reactants')
diagram.add_level(-5.4,'mlC1')
diagram.add_level(-15.6,'mlC2','last',) #Using 'last'  or 'l' it will be together with the previous level
diagram.add_level(28.5,'mTS1',color='g')
diagram.add_level(-9.7,'mCARB1')
diagram.add_level(-19.8,'mCARB2','l')
diagram.add_level(20,'mCARBX','last')

In [ ]:
def visualize_step(step):
    import re
    rxn = step.__repr__()
    components = rxn.split("<->")
    reactants = components[0].split("+")
    products = components[1].split("+")
    # keep only chars between parentheses
    reactants = [re.findall(r'\((.*?)\)', reactant) for reactant in reactants]
    products = [re.findall(r'\((.*?)\)', product) for product in products]
    # flatten list of lists
    reactants = [item for sublist in reactants for item in sublist]
    products = [item for sublist in products for item in sublist]
    rxn_string = " + ".join(reactants) + " -> " + " + ".join(products)
    diagram = ED()
    diagram.add_level(0, rxn_string.split(" -> ")[0])
    diagram.add_level(round(step.e_act[0], 2), 'TS', color='r')
    diagram.add_level(round(step.energy[0], 2), rxn_string.split(" -> ")[1])
    diagram.add_link(0,1)
    diagram.add_link(1,2)
    y = diagram.plot(ylabel="Energy / eV") # this is the default ylabel
    # add title in Arial and bold, shifted slightly above
    # the default position
    plt.title(rxn_string, fontname='Arial', fontweight='bold',
              y=1.05) 
    # from os import makedirs
    # from ase.io import write
    # from os.path import abspath
    # from matplotlib.offsetbox import OffsetImage, AnnotationBbox
    # makedirs('tmp', exist_ok=True)
    # for inter in step.reactants:        
    #     fig_path = abspath("tmp/{}.png".format(inter.code))
    #     write(fig_path, inter.molecule, show_unit_cell=0)
    #     arr_img = plt.imread(fig_path)
    #     im = OffsetImage(arr_img)
    #     ab = AnnotationBbox(im, (1, 0), xycoords='axes fraction')
    #     diagram.ax.add_artist(ab)
    # diagram.plot(ylabel="eV")
    return diagram
        
    

In [ ]:
y = rxn_net.visualize_reaction(10)

In [ ]:
y.fig.get_default_bbox_extra_artists()

In [ ]:
artists = y.fig.get_default_bbox_extra_artists()
type(artists[1])
# add rectangle to the y axis
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.colors import ListedColormap
import numpy as np

# create a list of Rectangle patches
rectangles = []
for i in range(10):
    rectangles.append(Rectangle((0, i), 1, 1))



In [ ]:
artists = y.fig.get_default_bbox_extra_artists()
rect_width = artists[2].get_position()[0] - artists[3].get_position()[0]
std = 0.3
rect_height = 1.96*2*std
anchor_point1 = (min(artists[2].get_position()[0], artists[3].get_position()[0]), )
anchor_point2 = (min(artists[2].get_position()[0], artists[3].get_position()[0]), )
anchor_point3 = (min(artists[2].get_position()[0], artists[3].get_position()[0]), )